# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Содержание
* [Изучение данных из файла](#Изучение-данных-из-файла)
* [Прeдобработка данных](#Предобработка-данных)
    * [Обработка пропусков](#Обработка-пропусков)
    * [Замена типов данных](#Замена-типов-данных)
    * [Обработка дубликатов](#Обработка-дубликатов)
    * [Лемматизация](#Лемматизация)
    * [Категоризация данных](#Категоризация-данных)
* [Ответы на вопросы](#Ответы-на-вопросы)
* [Общий вывод](#Общий-вывод)

## Изучение файла с данными

In [1]:
# импорт библиотеки pandas
import pandas as pd


In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/data.csv')

In [3]:
# получение первых 10 строк таблицы df
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов. В столбцах встречаются следующие типы данных:
float - в двух столбцах;
int - в пяти столбцах;
object - в пяти столбах.

Согласно документации к данным:

children — количество детей в семье;
days_employed — общий трудовой стаж в днях;
dob_years — возраст клиента в годах;
education — уровень образования клиента;
education_id — идентификатор уровня образования;
family_status — семейное положение;
family_status_id — идентификатор семейного положения;
gender — пол клиента;
income_type — тип занятости;
debt — имел ли задолженность по возврату кредитов;
total_income — ежемесячный доход;
purpose — цель получения кредита.

Анализ общей информации показал, что в наборе данных  столбцы 'days_employed'и 'total_income' содержат пропуски. Помимо указанных проблем с данными можно отметить наличие отрицательных значений в столбце 'days_employed', что не может соответствовать здравому смыслу.
Для исследования надежности заемщиков сперва необходимо провести предобработку данных.

## Предобработка данных

### Обработка пропусков

In [5]:
# нахождение количества пропущенных значений в таблице df
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
# вычисление среднего значения общего трудового стажа и медианы ежемесячного дохода
days_employed_mean = df['days_employed'].mean()
total_income_median = df['total_income'].median()

In [7]:
# заменим пропущенные значения в столбце 'days_employed' на средние значения, а в столбце 'total_income' - на медианные значения
df['days_employed'] = df['days_employed'].fillna(days_employed_mean)
df['total_income'] = df['total_income'].fillna(total_income_median)

In [8]:
# составим функцию, которая позволит сделать отрицательные значения положительными
def transformation (value):
    if value < 0:
        value *= -1
        return value
    else:
        return value

In [9]:
# применим функцию transformation к нашему набору данных и заменим отрицательные значения в столбце 'days_employed' на положительные
df['days_employed'] = df['days_employed'].apply(transformation)

In [10]:
df['days_employed'].abs()

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

Таким образом, была произведена замена отрицательных значений общего трудового стажа положительными значениями. Большое количество отрицательных чисел в данном столбце могло быть вызвано некорректной выгрузкой данных. Также были заменены пропуски в столбцах 'days_employed' и 'total_income' на среднее значение и медиану соответственно. Причиной пропусков, причем в одинаковом количестве и во взаимосвязанных столбцах, могло стать отсутствие официальной работы. Например, человек работает на себя.

### Замена типа данных

In [11]:
# изменим тип данных в столбцах 'days_employed' и 'total_income'
df = df.astype({'days_employed': 'int', 'total_income': 'int'})

В столбцах 'трудовой стаж' и 'ежемесячный доход' использовался тип данных float. С помощью метода astype была произведена замена на int.

### Обработка дубликатов

In [12]:
# получение уникальных значений в столбце 'education'
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [13]:
# получение уникальных значений в столбце 'family_status'
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [14]:
# получение уникальных значений в столбце 'income_type'
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [15]:
# получение уникальных значений в столбце 'purpose'
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [16]:
# получение уникальных значений в столбце 'children'
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [17]:
# приведем все символы базы данных к нижнему регистру для исключения дубликатов
df['education'] = df['education'].str.lower()

In [18]:
# в столбце с количеством детей заменим -1 на положительное значение, а 20 - на медианное, так как нет возможности определить,
# поставили к 2 лишний 0 или к 0 приписали 2
df['children'] = df['children'].replace(-1, 1)
children_median = df['children'].median()
df['children'] = df['children'].replace(20, children_median)

В наборе данных дубликаты встречаются в столбце "Образование". В написании уровня образования используются символы нижнего и верхнего регистра, что привело к дублурованию одиних и тех же слов. Причиной данной проблемы могла послужить некорректная выгрузка данных.
Также в процессе анализа данных были выявлены аномальные значения в столбце 'дети': -1 и 20. Первое значение было заменено на положительное, а второе - на медианное. 

### Лемматизация

In [19]:
# импорт библиотеки pymystem3 и collections
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [20]:
# на основании ранее изученных уникальных значений столбца 'purpose' выделим ключевые категории
categories = ['жилье', 'недвижимость', 'образование', 'свадьба', 'автомобиль']

In [21]:
# лемматизируем данные
def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in categories:
        if word in lemma:
            lemma = word
    return lemma

df['purpose_new'] = df['purpose'].apply(lemmatize)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [22]:
df['purpose_new'].value_counts()

недвижимость    6367
жилье           4473
автомобиль      4315
образование     4022
свадьба         2348
Name: purpose_new, dtype: int64

In [23]:
# заменим категорию "жилье" на "недвижимость"
df['purpose_new'] = df['purpose_new'].replace('жилье', 'недвижимость')
df['purpose_new'].value_counts()

недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_new, dtype: int64

С помощью лемматизации были выделены следующие основные цели, для которых брался кредит:
- недвижимость;
- автомобиль;
- образование;
- свадьба.
Такое разделение основано на выделении ключевых слов, которые встречаются в каждой цели в исходном виде. Поскольку жилье относится к недвижимости, оно было заменено на соответствующее значение.


### Категоризация данных

Проведем категоризацию данных по доходам заемщиков

In [24]:
# разобъем общий доход на квантили 25 %, 50 % И 75 %
statistics = df['total_income'].describe()
statistics[4:7]

25%    107798.0
50%    145017.0
75%    195543.0
Name: total_income, dtype: float64

In [25]:
# для категоризации доходов составим функцию
def determine_income_group(income):
    if income <= statistics[4]: 
        return 1
    elif statistics[4] < income <= statistics[5]: 
        return 2
    elif statistics[5] < income <= statistics[6]: 
        return 3
    else: 
        return 4

In [26]:
# применим функцию к столбцу доходов
df['income_group'] = df['total_income'].apply(determine_income_group)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,4
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,2
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,3
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,4
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,3


In [27]:
# проверим количество людей, принадлежащих той или иной категории в соответствии с ранее рассмотренными квантилями доходов
df['income_group'].value_counts()

2    6468
1    5382
4    5381
3    4294
Name: income_group, dtype: int64

Таким образом, доход 5382 заемщиков составляет менее или равен 107798 руб., доход 6468 - в диапозоне от 107798 руб. до 145017 руб., доход 4294 - в диапозоне от 145017 руб. до 195543 руб. и 5381 - более 195543 руб.

In [28]:
# для дальнейшего анализа добавим 2 категории: 1 - есть дети; 0 - детей нет
def children_exist(children):
    if children == 0:
        return 0
    else:
        return 1

In [29]:
# применим полученную функцию для категоризации детей
df['children_exist'] = df['children'].apply(children_exist)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new,income_group,children_exist
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,4,1
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,2,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,3,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,4,1
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,3,0


## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [30]:
# построим сводную таблицу
df_pivot = df.pivot_table(index = ['children_exist'], values = 'debt').round(3)
df_pivot

,debt
children_exist,
0,0.075
1,0.092


В итоге, если у человека есть ребенок, то вероятность задолженности составляет 9.2 %, если детей нет - 7.5 %. Данная ситуация может быть вызвана тем, что в случае наличия детей человек берет в кредит большую сумму и, соответственно, повышает вероятность не покрыть задолженность в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [31]:
# построим сводную таблицу
df_pivot = df.pivot_table(index = ['family_status'], values = ['debt']).round(3)
df_pivot

,debt
family_status,
Не женат / не замужем,0.097
в разводе,0.071
вдовец / вдова,0.066
гражданский брак,0.093
женат / замужем,0.075


Таким образом, люди, которые находятся в разводе, женаты/замужем или являются вдовцом/вдовой, с меньшей вероятностью не вернут долг банку, в то время как люди, которые не женаты/не замужем или находятся в гражданском браке, больше склонны не возвращать банковскую задолженность.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [32]:
# построим сводную таблицу
df_pivot = df.pivot_table(index = ['income_group'], values = ['debt']).round(3)
df_pivot

,debt
income_group,
1,0.079
2,0.085
3,0.089
4,0.072


Логично, что люди с самым высоким доходом с самой низкой вероятностью не вернут кредит в срок. Интересным является тот факт, что на втором месте находятся люди с наименьшим доходом. Это может быть вызвано нет, что в связи с ограниченным бюджетом они рациональнее распоряжаются своими деньгами. 

- Как разные цели кредита влияют на его возврат в срок?

In [33]:
# построим сводную таблицу
df_pivot = df.pivot_table(index = ['purpose_new'], values = ['debt']).round(3)
df_pivot

,debt
purpose_new,
автомобиль,0.093
недвижимость,0.072
образование,0.092
свадьба,0.079


Люди более ответственно подходят к возврату кредиторской задолженности, если они ее берут с целью приобретения недвижимости, так как, скорее всего, несвоевременное погашение кредита чревато ликвидацией недвижимости. С большей долей вероятности заемщики не вернут вовремя автокредит и кредит на образование.

## Общий вывод

Данные анализируемой выборки показали, что различные категории разных значений не в одинаковой степени влияют на возврат кредиторской задолженности в срок. Например, идеальным заемщик согласно данной выборки выглядит так: вдовец/вдова с доходом более 195543 руб. без детей, который/которая берет кредит на недвижимость. Вырисовывается картина ответственного человека, не обременного семейными проблемами с высоким доходом.